In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Cargar los datos (ajusta la ruta según sea necesario)
movies = pd.read_parquet('../Data/processed/movies_dataset.parquet')


# Paso 1: Preprocesamiento
# Crear una columna combinada de características
movies['genres'] = movies['genres'].astype(str)  # Asegurarse de que 'genres' es string
movies['title'] = movies['title'].astype(str)  # Asegurarse de que 'title' es string
movies['overview_clean'] = movies['overview_clean'].astype(str)  # Asegurarse de que 'overview_clean' es string

movies['combined_features'] = (
    movies['genres'].fillna('') + ' ' +
    movies['title'].fillna('') + ' ' +
    movies['overview_clean'].fillna('')
)

# Paso 2: Crear el modelo de similitud de contenido
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(movies['combined_features'])
cosine_sim = cosine_similarity(tfidf_matrix, dense_output=False)

# Paso 3: Función para obtener recomendaciones
def obtener_recomendaciones(titulo, cosine_sim=cosine_sim, movies=movies):
    # Verificar si la película existe en el DataFrame
    idx = movies[movies['title'].str.lower() == titulo.lower()].index
    if idx.empty:
        return {"error": "Película no encontrada"}
    
    idx = idx[0]  # Obtén el índice de la película
    sim_scores = list(enumerate(cosine_sim[idx].toarray().flatten()))  # Similaridades con todas las películas
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Ordenar por mayor similitud
    sim_scores = sim_scores[1:6]  # Seleccionar las 5 películas más similares
    
    movie_indices = [i[0] for i in sim_scores]
    recomendaciones = movies['title'].iloc[movie_indices].tolist()
    return {"recomendaciones": recomendaciones}

# Ejemplo de uso
print(obtener_recomendaciones("Toy Story"))


{'recomendaciones': ['Toy Story 2', 'Toy Reanimator', 'Toy Story of Terror!', 'Silent Night, Deadly Night 5: The Toy Maker', 'Toy Story 3']}
